# Chatbot speach to speach

## Configuracion

Primero, hay que asegurarse de estar usando un entorno de ejecución GPU para ejecutar este notebook (es necesario para que Whisper corra más rápido), de modo que la inferencia sea mucho más rápida. Si el siguiente comando falla, use el menú `entorno de ejecución` del menu y seleccione `Cambiar tipo de entorno ejecución`.

In [1]:
!nvidia-smi

Mon Nov 28 14:24:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
#@title Ejecutar esta celda para instalar las librerias necesarias y luego reiniciar el entorno de ejecución
from IPython.display import clear_output

!pip install git+https://github.com/openai/whisper.git 
!pip install jiwer gTTS sentencepiece sacremoses

clear_output()

In [2]:
#@title Ejecutar esta celda para cargar las librerias y modelos necesarios
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration, pipeline
import torch
import numpy as np
import whisper
import time

from gtts import gTTS
from google.colab.output import eval_js
from base64 import b64decode
import io
import ffmpeg

from scipy.io.wavfile import read as wav_read
from scipy.io.wavfile import write
from IPython.display import clear_output, Audio, display, HTML

# Cargamos whisper
wisper_model = whisper.load_model("medium")


# Cargamos el modelo para el chat
model_name = 'facebook/blenderbot-400M-distill'
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)


# Cargamos el traductor
english_model_name = "Helsinki-NLP/opus-mt-en-es"
translator_en_es = pipeline("translation", model=english_model_name)


100%|█████████████████████████████████████| 1.42G/1.42G [00:28<00:00, 53.0MiB/s]


Downloading:   0%|          | 0.00/127k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/802k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/826k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

In [3]:
#@title Ejecutar esta celda para cargar las funciones necesarias

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Presiona para comenzar a grabar");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Grabando... presione para detener";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Guardando la grabación... por favor espere!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

def get_text():
  """
  Aplicamos Whisper para obtener el texto desde un audio ingresado por el usuario
  """
  audio, sr = get_audio()
  name = 'record.wav'
  write(name, sr, audio)
  # Cargamos el audio
  audio = whisper.load_audio(name)

  # Le pasamos las tareas
  transcription = wisper_model.transcribe(audio) # por defecto task='transcribe'
  translation = wisper_model.transcribe(audio, task="translate")["text"]

  clear_output()

  print('Su transcripcion: {}\n'.format(transcription["text"]))
  # print("Idioma detectado: {}".format(transcription['language']))
  return translation

def get_speach(result):
  """
  A partir del resultado del chat en ingles, lo traducimos y pasamos a audio
  """
  spanish_text = translator_en_es(result)
  result_es = spanish_text[0]['translation_text']
  sound_file = 'output.wav'
  tts = gTTS(result_es, lang="es", tld='com.mx')
  tts.save(sound_file)
  wn = Audio(sound_file, autoplay=True)
  display(wn)

def chatbot():
  chat_on = True
  step = 0
  while chat_on:
      # Cargamos el audio y lo pasamos a texto en ingles para pasar al generador de texto condicional
      text_input = get_text()

      # codificamos la nueva entrada del usuario, agregar el eos_token y devolver un tensor en Pytorch
      new_user_input_ids = tokenizer.encode( text_input + tokenizer.eos_token, return_tensors='pt')

      # agregar los nuevos tokens de entrada de usuario al historial de chat
      bot_input_ids = torch.cat([torch.LongTensor(history), new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
      # Generar una respuesta mientras se limita el historial de chat total a 500 tokens,
      history = model.generate(bot_input_ids, 
                              max_length=500, 
                              pad_token_id=tokenizer.eos_token_id,  
                              temperature=0.3).tolist()
      # últimos tokens de salida del bot
      response = tokenizer.decode(history[0], skip_special_tokens=True)

      # pasamos el resultado a la funcion get_speach para obtener el audio
      get_speach(response)
      
      time.sleep(3)
      # Consultamos si se desea continuar
      desea_continuar = input("Desea continuar? Ingrese Y o N: ").lower()
      if desea_continuar=='n':
        chat_on = False
      step +=1

# ChatBot
Al ejecutar la siguiente linea automáticamente aparece un cuadro debajo indicando el inicio de la grabacion y comienza a grabar. Cuando se termina de hablar, presionar el cuadro para enviar. Le devolvera su grabacion y luego de unos instantes devuelve la transcripcion del audio enviado y posteriormente en audio la respuesta del chat. Si se desea continuar ingresar Y, de lo contrario ingresar N. 


In [4]:
#@title Ejecutar la siguiente linea de codigo para inicializar el chatbot
chatbot()

Su transcripcion:  Sí, deseo salir a comer unas pistas con mis amigos. ¿Vos tenés planes?



Desea continuar? Ingrese Y o N: N
